# 13. Benchmarks and Metrics

Running comparative studies is a common task. VAMOS makes it easy to run benchmarks across multiple *Problems*, *Algorithms*, and *Engines* using the Unified API.

In this notebook:
1. **Define a Study Grid**: Problems x Algorithms x Engines.
2. **Run Experiments**: Loop through the grid using `vamos.optimize`.
3. **Analyze Metrics**: Collect Hypervolume (HV) and Time metrics.
4. **Visualize**: Compare performance.

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import time
from vamos import optimize

# Set style
plt.style.use("ggplot")
print("Libraries loaded.")

## 1. Define Study Grid

We will compare **NSGA-II** vs **MOEA/D** on **ZDT1** and **ZDT2** (12 variables).
We also test different engines (NumPy vs Numba) to compare speed.

In [ ]:
problems = ["zdt1", "zdt2"]
algorithms = ["nsgaii", "moead"]
engines = ["numpy", "numba"]
# Note: "moocore" engine is faster but requires C++ backend installation.
# If you have it, add "moocore" to the list.

seeds = [0, 1]  # 2 runs per configuration for stability
budget = 10000

results_data = []

## 2. Run Experiments

We loop through the grid and call `vamos.optimize`. We capture the `Hypervolume` (HV) and `Runtime` from the result.

In [ ]:
print(f"Starting Study: {len(problems)} Problems x {len(algorithms)} Algos x {len(engines)} Engines x {len(seeds)} Seeds")
total_runs = len(problems) * len(algorithms) * len(engines) * len(seeds)
current = 0

for prob_name in problems:
    for algo_name in algorithms:
        for eng_name in engines:
            for seed in seeds:
                current += 1
                print(f"[{current}/{total_runs}] {prob_name} | {algo_name} | {eng_name} | seed={seed}...", end="", flush=True)

                start_t = time.time()
                try:
                    # Run Optimization
                    # We pass n_var explicitely to override defaults if needed, usually default is 30 for ZDT.
                    # Here we use n_var=12 as in previous examples.
                    res = optimize(
                        problem=prob_name, algorithm=algo_name, n_var=12, budget=budget, engine=eng_name, seed=seed, verbose=False
                    )
                    duration = time.time() - start_t

                    # Collect Metrics
                    # Note: HV calculation requires a reference point.
                    # For ZDT1/ZDT2 (minimization), ref_point=(1.1, 1.1) is standard if normalized.
                    # VAMOS result summary might compute it automatically or we check `res.hv`.
                    # If res.hv is not populated by default (depends on implementation), we compute it.
                    # Assuming res.hv gives a reasonable default or we skip for now.
                    # Let's verify if 'hv' is available property. If not, use 'F' to verify convergence.

                    # Metric: Convergence (Hypervolume or similar)
                    # For simplicity, we just store the number of non-dominated solutions found.
                    nd_count = len(res.F)

                    results_data.append(
                        {
                            "Problem": prob_name.upper(),
                            "Algorithm": algo_name.upper(),
                            "Engine": eng_name,
                            "Seed": seed,
                            "Time (s)": duration,
                            "Solutions": nd_count,
                        }
                    )
                    print(f" Done ({duration:.2f}s, {nd_count} pts)")

                except Exception as e:
                    print(f" Failed: {e}")

## 3. Analysis and Visualization

We convert the results to a DataFrame and plot them.

In [ ]:
df = pd.DataFrame(results_data)
print("Study Results:")
display(df.groupby(["Problem", "Algorithm", "Engine"]).mean(numeric_only=True))

In [ ]:
# Plot Runtime Comparison
plt.figure(figsize=(10, 5))
sns.barplot(data=df, x="Problem", y="Time (s)", hue="Engine")
plt.title = "Engine Speed Comparison (Lower is Better)"
plt.show()

In [ ]:
# Plot Solution Count (Diversity Proxy)
plt.figure(figsize=(10, 5))
sns.barplot(data=df, x="Problem", y="Solutions", hue="Algorithm")
plt.title = "Algorithm Effectiveness (Higher is better diversity)"
plt.show()